In [9]:
import numpy as np
import pandas as pd
import time
import cdsw
import cmlapi
import os
import json
import uuid

In [10]:
# if os.environ.get("MODEL_NAME") == "":
#     os.environ["MODEL_NAME"] = "LSTM-2"
# if os.environ.get("PROJECT_NAME") == "":
#     os.environ["PROJECT_NAME"] = "SDWAN"

In [11]:
# model_name = os.environ.get("MODEL_NAME")
# project_name = os.environ.get("PROJECT_NAME")

model_name = "lstm"
#model_name = "LSTM-2"
project_name = "SDWAN"

client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
target_model = client.list_all_models(search_filter=json.dumps({"name": model_name}))

proj_id = client.list_projects(search_filter=json.dumps({"name":project_name })).projects[0].id

mod_id = target_model.models[0].id

build_list = client.list_model_builds(project_id = proj_id, model_id = mod_id,sort='created_at')

build_id = build_list.model_builds[-1].id
model_deployments = client.list_model_deployments(project_id=proj_id,model_id=mod_id,build_id=build_id)
cr_number = model_deployments.model_deployments[0].crn

In [12]:
# write a function to print variable crn
metrics = cdsw.read_metrics(model_deployment_crn=cr_number,dev=False)

In [14]:
#metrics

In [16]:
#metrics['metrics'][0]

In [17]:
performance_metrics = []
core_metrics = []
for item in metrics['metrics']:
    if len(item['metrics'].keys()) < 5:
         performance_metrics.append(item['metrics'])
    else:
        core_metrics.append(item['metrics'])

In [18]:
core_metrics

[{'final_rx_gbs_label': [0.6087192349999999,
   10.110163449,
   3.542944531,
   1.609113628,
   8.76751056,
   1.797950447],
  'final_tx_gbs_label': [7.04093571,
   3.494632557,
   2.206972175,
   10.506076429,
   8.260370310999999,
   3.809892921],
  'forecast time': ['2024-05-23T13:00:00',
   '2024-05-23T14:00:00',
   '2024-05-23T15:00:00',
   '2024-05-23T16:00:00',
   '2024-05-23T17:00:00',
   '2024-05-23T18:00:00'],
  'input_rx_data': [6.332707443,
   2.75049255,
   1.783114251,
   0,
   1.031544663,
   0.8535213709999999,
   4.143807759,
   8.592050492,
   1.484060586,
   0.976592702,
   1.484060586,
   0.571406597,
   0.668427823,
   0.75274246,
   8.8223739,
   0,
   0.555023761,
   1.845765648,
   10.016815459,
   8.17313969,
   0.766537612,
   1.888395224,
   0.508248036,
   0.862901636,
   1.225272153],
  'input_tx_data': [16.858639016999998,
   3.418866322,
   5.5797604430000005,
   12.327977381,
   8.846197672,
   2.791354559,
   10.553354326,
   8.699411801,
   10.2867574

In [7]:
performance_metrics

[{'rmse': 5.562765954111612},
 {'rmse': 5.562765954111612},
 {'rmse': 4.927773960728153},
 {'rmse': 4.862253925303557},
 {'rmse': 4.966183900435357},
 {'rmse': 5.550734056256182},
 {'rmse': 5.1409435344527985},
 {'rmse': 5.550734056256182},
 {'rmse': 5.441778049730006},
 {'rmse': 5.4911251077007215}]

In [8]:
def check_last_rmse(values):
    if len(values) < 2:
        raise ValueError("The list must contain at least two values.")
    
    rmse_values = [item['rmse'] for item in values]
    last_value = rmse_values[-1]
    prior_values = rmse_values[:-1]
    average_prior = sum(prior_values) / len(prior_values)
    
    return last_value < average_prior

In [9]:
check_last_rmse(performance_metrics)

False

In [11]:
metrics_df = pd.json_normalize(core_metrics)

In [16]:
metrics_df.tail()

,final_rx_gbs_label,final_tx_gbs_label,forecast time,input_rx_data,input_tx_data,rx bytes prediction,tx bytes prediction
54,"[1.411229685, 1.305595741, 1.413211706, 10.531...","[10.606051833, 5.167073481, 2.53719506, 9.0066...","[2024-05-13T01:00:00, 2024-05-13T02:00:00, 202...","[1.273866347, 1.342958131, 8.140159838, 1.9908...","[4.014604812, 1.955984192, 2.886702137, 2.7588...","[0.8348015546798706, 0.8849822282791138, 0.918...","[0.057432450354099274, 0.0744861513376236, 0.0..."
55,"[1.094988562, 0.889843661, 3.747575266, 1.7418...","[1.43889315, 4.661999297, 11.812364196, 5.2579...","[2024-05-02T01:00:00, 2024-05-02T02:00:00, 202...","[2.465220145, 2.280574848, 1.70296266, 1.56591...","[9.689268193, 11.636575472, 3.688097012, 10.63...","[0.6253072023391724, 0.688372015953064, 0.7331...","[0.08840098977088928, 0.0815410241484642, 0.07..."
56,"[4.123924896, 9.623044791, 6.396868241, 1.7796...","[3.808545758, 2.429849325, 4.201446881, 3.0185...","[2024-05-17T01:00:00, 2024-05-17T02:00:00, 202...","[1.646113664, 0.567956557, 4.123836151, 2.1252...","[4.163275395, 2.391207379, 10.021471138, 5.015...","[0.5600473880767822, 0.614625096321106, 0.6599...","[0.06582796573638916, 0.06737610697746277, 0.0..."
57,"[9.711008288, 0.6623837619999999, 1.05802475, ...","[3.386650789, 7.865282364, 1.655461957, 5.5431...","[2024-05-08T01:00:00, 2024-05-08T02:00:00, 202...","[1.017495761, 6.484177698, 0.571594764, 8.3529...","[11.23759017, 1.6609205299999998, 11.11969373,...","[0.5358205437660217, 0.6750768423080444, 0.809...","[0.12858328223228455, 0.15729646384716034, 0.1..."
58,"[0.567956557, 4.123836151, 2.125209277, 1.0186...","[2.391207379, 10.021471138, 5.015414073, 3.850...","[2024-05-16T01:00:00, 2024-05-16T02:00:00, 202...","[1.290738923, 3.540820261, 0.855667453, 3.8402...","[2.422788214, 4.430113012, 4.592582602, 13.240...","[0.4909769892692566, 0.6044356226921082, 0.665...","[0.0815158262848854, 0.09619943797588348, 0.11..."


In [15]:
metrics_df.isnull().sum()

final_rx_gbs_label     1
final_tx_gbs_label     1
forecast time          0
input_rx_data          0
input_tx_data          0
rx bytes prediction    0
tx bytes prediction    0
dtype: int64

In [19]:
metrics_df[metrics_df.isna().any(axis=1)]

,final_rx_gbs_label,final_tx_gbs_label,forecast time,input_rx_data,input_tx_data,rx bytes prediction,tx bytes prediction
27,NaN,NaN,"[2024-05-31T01:00:00, 2024-05-31T02:00:00, 202...","[9.142250296, 11.37641808, 1.391518371, 1.6188...","[13.741614516, 1.444370114, 4.774979732, 11.43...","[0.4038369059562683, 0.4840385317802429, 0.551...","[0.10492565482854843, 0.09039974957704544, 0.1..."
